In [33]:
import glob
import mne
import numpy as np
import pandas as pd
import torch
import pickle



# list files
files_name_eeglab = sorted(glob.glob('./face_processing_data/*/eeg/sub-*_task-FaceRecognition_eeg.set'))
files_name_tsv = sorted(glob.glob('./face_processing_data/sub-*/eeg/sub-*_task-FaceRecognition_events.tsv'))



###############################################################

def get_onset_duration(df):
    """ This function gets the onset and duration given the df """
    onset = []
    duration = []
    trial_type = []
    label = []

    for i in range(len(df)):
        if df['event_type'][i] == 'button_press':
            if df['event_type'][i-1] == 'faces':
                onset.append(df['onset'][i-1])
                duration.append(df['onset'][i] - df['onset'][i-1])
                trial_type.append(df['trial_type'][i-1])
                label.append(df['value'][i-1])

                # code to get the max duration
                # this allows me to get a size 
                # of the window without overlaps
                # of 1.4 seconds
                # try:
                #     duration_max.append(df['onset'][i+1] - df['onset'][i-1])
                # except:
                #     pass


    return np.array(onset), np.array(duration), np.array(trial_type), np.array(label) # , np.array(duration_max)

###############################################################

def get_data_eeglab(files_name_eeglab, files_name_tsv, base_path):
    """ This function loads the data from the eeglab files and the tsv 
    files and saves them in a npz file.

    @param files_name_eeglab: list of strings
        list of the eeglab files
    @param files_name_tsv: list of strings
        list of the tsv files
    @param base_path: string
        path where to save the data

    @return: None
    """
    files_name = zip(files_name_eeglab, files_name_tsv)

    # target labels
    target_labels = torch.tensor([])
    target_trial_type = np.array([])
    X = torch.tensor([])
    event_dict_all = []
    subject_id = torch.tensor([])

    data = []
    for eeglab_file, tsv in files_name:

        #######################################################
        ######## LOAD EEG DATA FROM THE EEG FILE ##############
        #######################################################
        # load the data
        raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)


        #######################################################
        ######## PERFORM ICA ##################################
        #######################################################

        # perform ica and plot
        # ica = mne.preprocessing.ICA(n_components=0.95, random_state=97, max_iter='auto')
        # ica.fit(raw)

        # ica_raw = ica.get_components()

        #######################################################
        ######## LOAD ANNOTATIONS FROM THE TSV FILE ###########
        #######################################################

        df = pd.read_csv(tsv, sep='\t')

        # get the onset and duration
        onset, duration, trial_type, label = get_onset_duration(df)

        # duration_max = np.max(duration) # 1.4s

        # get the target labels
        target_labels = torch.cat([target_labels, torch.from_numpy(label)])
        target_trial_type = np.concatenate([target_trial_type, trial_type])

        # add the annotations to the raw data
        annot = mne.Annotations(
            onset=list(onset),  # in seconds
            duration=[1.4]*onset.shape[0],  # in seconds, too
            description=list(trial_type),
            # id_code=list(df['value']),
        )

        # print the annotations
        # print(annot)

        raw2 = raw.copy().set_annotations(annot)

        # raw2.plot()
        # print(pd.DataFrame(raw2.annotations))

        # get events from the annotations
        events_from_annot, event_dict = mne.events_from_annotations(raw2)

        # epochs
        epochs = mne.Epochs(raw2, events_from_annot, event_id=event_dict, tmin=-0.2, tmax=0.55, preload=True)

        # filter the data
        epochs.filter(4, 80)

        # apply baseline correction
        epochs.apply_baseline((None, 0))

        # # plot the epochs
        # epochs.plot(n_epochs=10, n_channels=10)

        # subject id
        subject_id = torch.cat([subject_id,torch.tensor([int(i) for i in [eeglab_file.split('/')[2].split('-')[1]]])])

        # concatenate the data
        X = torch.cat([X, torch.from_numpy(epochs.get_data()).to(torch.float32)])

        # concatenate the event dict
        event_dict_all.append(event_dict)

        # Save the data
        data.append((eeglab_file.split('/')[2], epochs.get_data(), epochs.info, epochs.events, epochs.event_id, event_dict, epochs.tmin, epochs.tmax))

        for d in data:
            print('SAVED: '+base_path + 'data_{}.npz'.format(d[0]))
            np.savez_compressed(base_path + 'data_{}.npz'.format(d[0]), data=np.array(d[1],dtype='float32'))

        # # save the data
        torch.save(X, base_path + '/dataset/X.pt')
        torch.save(target_labels, base_path + '/dataset/target_labels.pt')
        # torch.save(target_trial_type, base_path + '/dataset/target_trial_type.pt')
        np.save(base_path + '/dataset/target_trial_type.npy', target_trial_type)
        # torch.save(subject_id, base_path + '/dataset/subject_id.pt')
        with open(base_path + '/dataset/event_dict_all.pkl', 'wb') as f:
            pickle.dump(event_dict_all, f)
        

base_path = './data/face_recognition_preprocessed_data/'
get_data_eeglab(files_name_eeglab, files_name_tsv, base_path)

# import pickle
# with open('./data/face_recognition_preprocessed_data/face_recognition_preprocessed_data.pkl', 'wb') as f:
#     pickle.dump(data, f)


/tmp/ipykernel_622/4190618924.py:75: RuntimeWarning: Unknown types found, setting as type EEG:
ekg: ['EEG063', 'EEG064']
heog: ['EEG061']
veog: ['EEG062']
  raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)
/tmp/ipykernel_622/4190618924.py:75: RuntimeWarning: Estimated head radius (1.1 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)


Used Annotations descriptions: ['famous_new', 'famous_second_early', 'famous_second_late', 'scrambled_new', 'scrambled_second_early', 'scrambled_second_late', 'unfamiliar_new', 'unfamiliar_second_early', 'unfamiliar_second_late']
Not setting metadata
584 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 584 events and 189 original time points ...


/tmp/ipykernel_622/4190618924.py:75: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)


0 bad epochs dropped
Setting up band-pass filter from 4 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 413 samples (1.652 s)



/tmp/ipykernel_622/4190618924.py:126: RuntimeWarning: filter_length (413) is longer than the signal (189), distortion is likely. Reduce filter length or filter a longer signal.
  epochs.filter(4, 80)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


...

0 bad epochs dropped
Setting up band-pass filter from 4 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 413 samples (1.652 s)



/tmp/ipykernel_622/4190618924.py:126: RuntimeWarning: filter_length (413) is longer than the signal (189), distortion is likely. Reduce filter length or filter a longer signal.
  epochs.filter(4, 80)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Applying baseline correction (mode: mean)


[Parallel(n_jobs=1)]: Done 64602 out of 64602 | elapsed:   11.2s finished


SAVED: ./data/face_recognition_preprocessed_data/data_sub-002.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-003.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-004.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-005.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-006.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-007.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-008.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-009.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-010.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-011.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-012.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-013.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-014.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-015.npz
SAVED: ./data/face_recognition_preprocessed_data/data_sub-016.npz
SAVED: ./d

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv('./face_processing_data/sub-002/eeg/sub-002_task-FaceRecognition_events.tsv', sep='\t')

def get_onset_duration(df):
    """ This function gets the onset and duration given the df """
    onset = []
    duration = []
    trial_type = []
    label = []

    for i in range(len(df)):
        if df['event_type'][i] == 'button_press':
            if df['event_type'][i-1] == 'faces':
                onset.append(df['onset'][i-1])
                duration.append(df['onset'][i] - df['onset'][i-1])
                trial_type.append(df['trial_type'][i-1])
                label.append(df['value'][i-1])

                # code to get the max duration
                # this allows me to get a size 
                # of the window without overlaps
                # of 1.4 seconds
                # try:
                #     duration_max.append(df['onset'][i+1] - df['onset'][i-1])
                # except:
                #     pass


    return np.array(onset), np.array(duration), np.array(trial_type), np.array(label) # , np.array(duration_max)

onset, duration, trial_type, label = get_onset_duration(df)

print(duration.max(), duration.mean(), duration.min()) # 1.4, 0.7, 0.2 # we will use 1.419s

1.4190909000000147 0.7127023756849297 0.13363699999990786


In [30]:
torch.tensor([int(i) for i in [files_name_eeglab[0].split('/')[2].split('-')[1]]*10])

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
df = pd.read_csv('./face_processing_data/sub-002/eeg/sub-002_task-FaceRecognition_events.tsv', sep='\t')

df

,onset,duration,sample,trial_type,response_time,stim_file,value,face_type,event_type,event_order,trial_dist,time_dist
0,24.205818,NaN,NaN,unfamiliar_new,NaN,u032.bmp,13,unfamiliar,faces,1,0,0.0000
1,25.154000,NaN,NaN,left_nonsym,NaN,NaN,256,NaN,button_press,0,0,0.0000
2,27.245818,NaN,NaN,unfamiliar_second_early,NaN,u032.bmp,14,unfamiliar,faces,2,1,3.0400
3,27.893091,NaN,NaN,left_nonsym,NaN,NaN,256,NaN,button_press,0,0,0.0000
4,30.353091,NaN,NaN,unfamiliar_new,NaN,u088.bmp,13,unfamiliar,faces,1,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
1474,2975.132182,NaN,NaN,famous_new,NaN,f002.bmp,5,famous,faces,1,0,0.0000
1475,2975.833091,NaN,NaN,right_sym,NaN,NaN,4096,NaN,button_press,0,0,0.0000
1476,2978.239455,NaN,NaN,famous_second_early,NaN,f002.bmp,6,famous,faces,2,1,3.1073
1477,2978.732182,NaN,NaN,right_sym,NaN,NaN,4096,NaN,button_press,0,0,0.0000


In [1]:

import torch

base_path = './data/face_recognition_preprocessed_data/'
X = torch.load(base_path + '/dataset/X.pt')

X.shape


torch.Size([15392, 74, 189])

In [4]:
import glob
import mne
import numpy as np
import pandas as pd
import torch
import pickle



# list files
files_name_eeglab = sorted(glob.glob('./face_processing_data/*/eeg/sub-*_task-FaceRecognition_eeg.set'))
files_name_tsv = sorted(glob.glob('./face_processing_data/sub-*/eeg/sub-*_task-FaceRecognition_events.tsv'))



def get_onset_duration(df):
    """ This function gets the onset and duration given the df """
    onset = []
    duration = []
    trial_type = []
    label = []

    for i in range(len(df)):
        if df['event_type'][i] == 'button_press':
            if df['event_type'][i-1] == 'faces':
                onset.append(df['onset'][i-1])
                duration.append(df['onset'][i] - df['onset'][i-1])
                trial_type.append(df['trial_type'][i-1])
                label.append(df['value'][i-1])

                # code to get the max duration
                # this allows me to get a size 
                # of the window without overlaps
                # of 1.4 seconds
                # try:
                #     duration_max.append(df['onset'][i+1] - df['onset'][i-1])
                # except:
                #     pass


    return np.array(onset), np.array(duration), np.array(trial_type), np.array(label) # , np.array(duration_max)


def get_data_eeglab(files_name_eeglab, files_name_tsv, base_path):
    """ 
    """
    files_name = zip(files_name_eeglab, files_name_tsv)

    for eeglab_file, tsv in files_name:

        #######################################################
        ######## LOAD EEG DATA FROM THE EEG FILE ##############
        #######################################################
        # load the data
        raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)

        df = pd.read_csv(tsv, sep='\t')

        # get the onset and duration
        onset, duration, trial_type, label = get_onset_duration(df)

        # duration_max = np.max(duration) # 1.4s


        # add the annotations to the raw data
        annot = mne.Annotations(
            onset=list(onset),  # in seconds
            duration=[1.4]*onset.shape[0],  # in seconds, too
            description=list(trial_type),
            # id_code=list(df['value']),
        )

        # print the annotations
        # print(annot)

        raw2 = raw.copy().set_annotations(annot)

        # raw2.plot()
        # print(pd.DataFrame(raw2.annotations))

        # get events from the annotations
        events_from_annot, event_dict = mne.events_from_annotations(raw2)

        # epochs
        epochs = mne.Epochs(raw2, events_from_annot, event_id=event_dict, tmin=-0.2, tmax=0.55, preload=True)

        # filter the data
        epochs.filter(4, 80)

        # apply baseline correction
        epochs.apply_baseline((None, 0))

        print(raw.info)
        print(epochs.info)
        print(epochs.times)
        break



base_path = './data/face_recognition_preprocessed_data/'
get_data_eeglab(files_name_eeglab, files_name_tsv, base_path)

/tmp/ipykernel_282/246143246.py:55: RuntimeWarning: Unknown types found, setting as type EEG:
ekg: ['EEG063', 'EEG064']
heog: ['EEG061']
veog: ['EEG062']
  raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)
/tmp/ipykernel_282/246143246.py:55: RuntimeWarning: Estimated head radius (1.1 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)


Used Annotations descriptions: ['famous_new', 'famous_second_early', 'famous_second_late', 'scrambled_new', 'scrambled_second_early', 'scrambled_second_late', 'unfamiliar_new', 'unfamiliar_second_early', 'unfamiliar_second_late']
Not setting metadata
584 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 584 events and 189 original time points ...


/tmp/ipykernel_282/246143246.py:55: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)


0 bad epochs dropped
Setting up band-pass filter from 4 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 413 samples (1.652 s)



/tmp/ipykernel_282/246143246.py:88: RuntimeWarning: filter_length (413) is longer than the signal (189), distortion is likely. Reduce filter length or filter a longer signal.
  epochs.filter(4, 80)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Applying baseline correction (mode: mean)
<Info | 8 non-empty values
 bads: []
 ch_names: EEG001, EEG002, EEG003, EEG004, EEG005, EEG006, EEG007, EEG008, ...
 chs: 74 EEG
 custom_ref_applied: False
 dig: 77 items (3 Cardinal, 74 EEG)
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 74
 projs: []
 sfreq: 250.0 Hz
>
<Info | 8 non-empty values
 bads: []
 ch_names: EEG001, EEG002, EEG003, EEG004, EEG005, EEG006, EEG007, EEG008, ...
 chs: 74 EEG
 custom_ref_applied: False
 dig: 77 items (3 Cardinal, 74 EEG)
 highpass: 4.0 Hz
 lowpass: 80.0 Hz
 meas_date: unspecified
 nchan: 74
 projs: []
 sfreq: 250.0 Hz
>
[-0.2   -0.196 -0.192 -0.188 -0.184 -0.18  -0.176 -0.172 -0.168 -0.164
 -0.16  -0.156 -0.152 -0.148 -0.144 -0.14  -0.136 -0.132 -0.128 -0.124
 -0.12  -0.116 -0.112 -0.108 -0.104 -0.1   -0.096 -0.092 -0.088 -0.084
 -0.08  -0.076 -0.072 -0.068 -0.064 -0.06  -0.056 -0.052 -0.048 -0.044
 -0.04  -0.036 -0.032 -0.028 -0.024 -0.02  -0.016 -0.012 -0.008 -0.004
  0.     0.004  

[Parallel(n_jobs=1)]: Done 43216 out of 43216 | elapsed:    8.3s finished
